<a href="https://colab.research.google.com/github/wing787/books-python-satellite-data-analysis-basic/blob/main/2_use_STAC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab利用時には以下のコマンドを実行する（初回のみ）←実行済み
# !apt-get install -y libspatialindex-dev

In [ ]:
# STACを利用するために、あらためてライブラリのインストールを行う。
# Colab利用時にはインストール後ランタイムを再起動する。（放置していて、ランタイム接続が切れていれば必ず再度実行）
# !pip install cartopy
# !pip install pygeos
# !pip install rtree
# !pip install sat-search
# !pip install rasterio
# !pip install pystac-client

In [ ]:
import os, json
from shapely.geometry import MultiPolygon, Polygon, box
from fiona.crs import from_epsg
import numpy as np
from pystac_client import Client
from satsearch import Search
from io import BytesIO
import urllib
from PIL import Image
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import warnings
import rasterio as rio
from rasterio.mask import mask
from rasterio.plot import show
from osgeo import gdal
warnings.filterwarnings('ignore')

print("done")

In [ ]:
AREA = [
    [
        139.708159,
        35.6593884
    ],
    [
        139.7067857,
        35.4817801
    ],
    [
        139.8619676,
        35.4817801
    ],
    [
        139.8578477,
        35.6493456
    ],
    [
        139.708159,
        35.6593884
    ]
]

# 左回りもしくは右回りに対応。
# 地図を左に大きく動かして場所を選択したり、右側に何周かして場所を選択したりすると、上記の経度が-180や、+180度を超えたりする。
# その際には、下のコードを実行する。
for i in range(len(AREA)):
    if AREA[i][0] >= 0:
        AREA[i][0] = AREA[i][0]%360
    else:
        AREA[i][0] = -(abs(AREA[i][0]%360) + 360)

In [ ]:
# AREAから最小緯度・経度、最大緯度・経度を取得する。
areaLon = []
areaLat = []
# iterating each number in list
for coordinate in AREA:
    areaLon.append(coordinate[0])
    areaLat.append(coordinate[1])

minLon = np.min(areaLon)  # min longitude
maxLon = np.max(areaLon)  # max longitude
minLat = np.min(areaLat)  # min latitude
maxLat = np.max(areaLat)  # max latitude

# 取得範囲を指定するための関数を定義
def selSquare(lon, lat, delta_lon, delta_lat):
  c1 = [lon + delta_lon, lat + delta_lat]
  c2 = [lon + delta_lon, lat - delta_lat]
  c3 = [lon - delta_lon, lat + delta_lat]
  c4 = [lon - delta_lon, lat - delta_lat]
  geometry = {'type': 'Polygon', 'coordinates': [[c1, c2, c3, c4, c1]]}
  return geometry

In [ ]:
bbox = selSquare(minLon, minLat, maxLon, maxLat)
dates = '2021-04-10/2021-06-10'  # '20210410', '20210610'



API_URL = 'https://earth-search.aws.element84.com/v0'
COLLECTION = "sentinel-s2-l2a-cogs"  # Sentinel-2, Level 2A (BOA)
s2STAC = Client.open(API_URL, headers=[])


s2Search = s2STAC.search (
   intersects = bbox,
   datetime = dates,
   query = {"eo:cloud_cover": {"lt": 20}, "sentinel:valid_cloud_cover": {"eq": True}},
   collections = COLLECTION)

s2_items = [i.to_dict() for i in s2Search.get_items()]
print(f"{len(s2_items)} のシーンを取得")

In [ ]:
items = s2Search.get_all_items()
gf = gpd.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")

In [ ]:
# 雲量で並び替え
gfSroted = gf.sort_values('eo:cloud_cover').reset_index(drop=True)
gfSroted.head()

In [ ]:
# 最も雲の量が少ないシーンを選択し、サムネイル画像も取得する関数を定義します。
def sel_items(scene_items, product_id):
  item = [x.assets for x in scene_items\
         if x.properties['sentinel:product_id'] == product_id]
  thumbUrl = [x.assets['thumbnail'].href for x in scene_items\
             if x.properties['sentinel:product_id'] == product_id]
  return item, thumbUrl

selected_item, thumbUrl = sel_items(items, gfSorted['sentinel:product_id'][0])
print(thumbUrl)
print(selected_item)

thumbImg = Image.open(BytesIO(requests.get(thumbUrl[0]).content))
plt.figure(figsize=(5,5))
plt.imshow(thumbImg)

In [ ]:
def getPreview(thumbList, r_num, c_num, figsize_x=12, figsize_y=12):
  """
  サムネイル画像を指定した引数に応じて行列状に表示
  r_num: 行数
  c_num: 列数
  """
  thumbs = thumbList
  f, ax_list = plt.subplots(r_num, c_num, figsize=(figsize_x, figsize_y))
  for row_num, ax_row in enumerate(ax_list):
    for col_num, ax in enumerate(ax_row):
      if len(thumbs) < r_num * c_num:
        len_shortrage = r_num * c_num - len(thumbs)  # 行列の不足分を算出
        count = row_num * c_num + col_num
        if count < len(thumbs):
          ax.label_outer()  # サブプロットのタイトルと、軸のタイトルが重複しないようにする
          ax.imshow(io.imread(thumbs[row_num * c_num + col_num]))
          ax.set_title(thumbs[row_num * c_num + col_num][60:79])
        else:
          for i in range(len_shortrage):
            blank = np.zeros([100, 100, 3], dtype=np.unit8)
            blank.fill(255)
            ax.label_outer()
            ax.imshow(blank)
      else:
        ax.label_outer()
        ax.imshow(io.imread(thumbs[row_num * c_num + col_num]))
        ax.set_title(thumbs[row_num * c_num + col_num][60:79])

  return plt.show()

In [ ]:
allthumburl = [catalog[gfSroted.id[i]]['thumbnail'].urlpath for i in range(12)]
getPreview(allthumburl[0:13], 3, 4, 16, 16)

In [ ]:
# 雲量が最も少ない画像を取得
item = catalog[gfSroted.id[4]]

# サムネイル画像の表示（RGB画像）
Image(item['thumbnail'].urlpath)

In [ ]:
bandLists = ['B04', 'B03', 'B02']
file_url = [item[band].metadata['href'] for band in bandLists]
print(file_url)

In [ ]:
# 画像の作成フォルダ作成
folder_name = 'sentinel2COG'
os.makedirs(folder_name, exist_ok=True)
RGB_dir = 'content/' + folder_name

In [ ]:
# 参照：https://automating-gis-processes.github.io/CSC18/lessons/L6/clipping-raster.html

bbox = box(minLon, minLat, maxLon, maxLat)
geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))
geo = geo.to_crs(crs='epsg:32654')  # Sentinel-2の画像に合わせる

def getFeatures(gdf):
  """rasterioで読み取れる形のデータへ変換するための関数"""
  return [json.loads(gdf.to_json())['features'][0]['geometry']]


coords = getFeatures(geo)
print(coords)

In [ ]:
# read image
b2 = rio.open(file_url[2])
b3 = rio.open(file_url[1])
b4 = rio.open(file_url[0])

# 出力ファイル名
RGB_path = os.path.join(RGB_dir, 'sentinel-2_l2a-cogs' + '.tif')

# 念の為
if not os.path.exists(RGB_dir):
    os.makedirs(RGB_dir)

# GeoTIFFの作成
RGB_color = rio.open(RGB_path,'w',driver='Gtiff', # driverにGtiff(GeoTIFF)
                    width=b4.width, height=b4.height, # 画像の高さや幅を指定。B04のバンドと同じ大きさにしています
                    count=3, # 3つのバンドを利用 (B02, B03, B04)
                    crs=b4.crs, # crsもB04と同様。epsg:32654
                    transform=b4.transform, # データに対する変換も同様のもの
                    dtype=rio.uint16 # データ型を指定
                    )
# 各々のバンド情報をRGB_colorに書き込み
RGB_color.write(b2.read(1), 3)  # 青
RGB_color.write(b3.read(1), 2)  # 緑
RGB_color.write(b4.read(1), 1)  # 赤
RGB_color.close()

with rio.open(RGB_path) as src:
  out_image, out_transform = rio.mask.mask(src, coords, crop=True)  # mask処理の実行
  out_meta = src.meta  # 作成する画像の情報はもともとの画像と同様のものにする

# メタ情報の更新
out_meta.update({'driver': 'GTiff',
                'height': out_image.shape[1],
                'width': out_image.shape[2],
                'transform': out_transform})


# 画像の書き出し
with rio.open(RGB_path, 'w', **out_meta) as dest:
  dest.write(out_image)

# 画像表示のため8bit形式で書き出し、画像の色味も調整する
scale = '-scale 0 255 0 25'
options_list = ['-ot Byte', '-of Gtiff', scale]
options_string = ' '.join(options_list)

gdal.Translate(os.path.join(RGB_dir, 'sentinel-2_l2a-cogs_Masked' + '.tif'), os.path.join(RGB_dir, 'sentinel-2_l2a-cogs' + '.tif'), options = options_string)

print('done')

In [ ]:
plt.figure(figsize=(6, 10))
RGB_2017octCOG = rio.open('./content/sentinel2COG/sentinel-2_l2a-cogs_Masked.tif')
show(RGB_2017octCOG.read([1,2,3]))